###Задание на практическую работу
Написать код для подключения к БД на джино через питон.
Автоматизировать исполнение основных SQL запросов
(Селект, апдейт, инсерт, создание и удаление таблицы, экспорт и импорт из цсв)
загрузить тестовые данные, протестировать все функции.

Первым делом мы восстановили все файлы и БД на джино.

Теперь подключимся к MySQL базе данных на хостинге Джино с использованием SQLAlchemy. Выполним тестовый запрос SELECT 1 для проверки подключения.

In [14]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

DB_HOST = "mysql.65e3ab49565f.hosting.myjino.ru"
DB_PORT = 3306
DB_USER = "j30084097"
DB_PASS = quote_plus("7f9vGAxSu")
DB_NAME = "j30084097"

DATABASE_URL = (
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    "?charset=utf8mb4"
)

engine = create_engine(
    DATABASE_URL,
    pool_pre_ping=True,
    pool_recycle=1800,
    # Если панель требует SSL, раскомментируйте:
    # connect_args={"ssl": {"ssl_disabled": False}}
)

# Проверка соединения
with engine.connect() as conn:
    print(conn.execute(text("SELECT 1")).scalar())

1


Установим библиотеку  PyMySQL

In [15]:
%pip install pymysql

Создадим полную систему для работы с БД. Реализуес все основные SQL операции. Автоматически протестируем все функции.

In [16]:
# ==== ОПЕРАЦИИ С БД (engine уже определён в твоём коде выше) ====
from __future__ import annotations

import csv, os
from typing import Iterable, Dict, Any, List, Optional

from sqlalchemy import (
    MetaData, Table, Column,
    Integer, String, Float, Boolean, DateTime,
    text, select, func
)
from sqlalchemy.engine import Engine
from sqlalchemy.exc import SQLAlchemyError

# ----- схема таблицы (пример: students) -----
metadata = MetaData()

students = Table(
    "students",
    metadata,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("name", String(100), nullable=False),
    Column("age", Integer, nullable=False),
    Column("avg_grade", Float, nullable=False),
    Column("enrolled", Boolean, nullable=False, default=True),
    Column("created_at", DateTime, server_default=func.current_timestamp()),
)

# ----- базовые операции -----
def check_connection(engine: Engine) -> None:
    with engine.connect() as conn:
        one = conn.scalar(text("SELECT 1"))
        version = conn.scalar(text("SELECT VERSION()"))
        dbname = conn.scalar(text("SELECT DATABASE()"))
        print(f"SELECT 1 -> {one} | VERSION() -> {version} | DB -> {dbname}")

def create_table(engine: Engine) -> None:
    metadata.create_all(engine, tables=[students])
    print("Таблица students создана (если отсутствовала)")

def drop_table(engine: Engine) -> None:
    with engine.begin() as conn:
        conn.execute(text("DROP TABLE IF EXISTS students"))
    print("Таблица students удалена (если была)")

def insert_rows(engine: Engine, rows: Iterable[Dict[str, Any]]) -> int:
    rows = list(rows)
    if not rows:
        return 0
    with engine.begin() as conn:
        res = conn.execute(students.insert(), rows)
    n = res.rowcount if res.rowcount is not None else len(rows)
    print(f" Вставлено строк: {n}")
    return n

def update_rows(engine: Engine, where: Dict[str, Any], updates: Dict[str, Any]) -> int:
    stmt = students.update()
    for k, v in where.items():
        stmt = stmt.where(getattr(students.c, k) == v)
    stmt = stmt.values(**updates)
    with engine.begin() as conn:
        res = conn.execute(stmt)
    n = res.rowcount or 0
    print(f" Обновлено строк: {n}")
    return n

def select_rows(engine: Engine, where: Optional[Dict[str, Any]] = None, limit: Optional[int] = None) -> List[Dict[str, Any]]:
    stmt = select(students)
    if where:
        for k, v in where.items():
            stmt = stmt.where(getattr(students.c, k) == v)
    if limit:
        stmt = stmt.limit(limit)
    with engine.connect() as conn:
        rows = conn.execute(stmt).mappings().all()
    out = [dict(r) for r in rows]
    print(f"Получено строк: {len(out)}")
    return out

def export_to_csv(engine: Engine, path: str) -> str:
    rows = select_rows(engine)
    cols = [c.name for c in students.columns] if not rows else list(rows[0].keys())
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=cols)
        w.writeheader()
        for r in rows:
            w.writerow(r)
    print(f"Экспортировано в CSV: {path}")
    return path

def import_from_csv(engine: Engine, path: str) -> int:
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    buf: List[Dict[str, Any]] = []
    with open(path, "r", newline="", encoding="utf-8") as f:
        r = csv.DictReader(f)
        for row in r:
            buf.append({
                "name": row.get("name") or row.get("Name"),
                "age": int(row.get("age") or row.get("Age")),
                "avg_grade": float(row.get("avg_grade") or row.get("Avg_Grade")),
                "enrolled": str(row.get("enrolled") or row.get("Enrolled", "1")).lower() in ("1","true","t","yes","y")
            })
    print(f"Импорт из CSV: {len(buf)} строк")
    return insert_rows(engine, buf)

# ----- автотест всех функций -----
def run_all(engine: Engine, drop_at_end: bool = False) -> None:
    try:
        print("=== Проверка подключения ===")
        check_connection(engine)

        print("\n=== DROP/CREATE ===")
        drop_table(engine)
        create_table(engine)

        print("\n=== INSERT (тестовые данные) ===")
        insert_rows(engine, [
            {"name": "Alice",   "age": 19, "avg_grade": 4.2, "enrolled": True},
            {"name": "Bob",     "age": 21, "avg_grade": 3.8, "enrolled": True},
            {"name": "Charlie", "age": 20, "avg_grade": 4.9, "enrolled": False},
        ])

        print("\n=== SELECT (все) ===")
        for r in select_rows(engine):
            print(r)

        print("\n=== UPDATE (Bob → avg_grade=4.1) ===")
        update_rows(engine, where={"name": "Bob"}, updates={"avg_grade": 4.1})
        print(select_rows(engine, where={"name": "Bob"}))

        print("\n=== Экспорт в CSV ===")
        export_path = os.path.abspath("students_export.csv")
        export_to_csv(engine, export_path)

        print("\n=== Импорт из CSV (создадим файл и загрузим) ===")
        import_path = os.path.abspath("students_import.csv")
        with open(import_path, "w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=["name","age","avg_grade","enrolled"])
            w.writeheader()
            w.writerow({"name":"Diana","age":22,"avg_grade":4.3,"enrolled":1})
            w.writerow({"name":"Egor","age":18,"avg_grade":3.7,"enrolled":1})
        import_from_csv(engine, import_path)

        print("\n=== COUNT ===")
        with engine.connect() as conn:
            total = conn.scalar(select(func.count()).select_from(students))
        print("Всего записей:", total)

        if drop_at_end:
            print("\n=== DROP (по завершении) ===")
            drop_table(engine)

        print("\n Все операции завершены")

    except SQLAlchemyError as e:
        print("Ошибка SQLAlchemy:", repr(e))
    except Exception as e:
        print(" Ошибка:", repr(e))

# Запуск:
run_all(engine, drop_at_end=False)


=== Проверка подключения ===
SELECT 1 -> 1 | VERSION() -> 10.11.14-MariaDB-cll-lve-log | DB -> j30084097

=== DROP/CREATE ===
Таблица students удалена (если была)
Таблица students создана (если отсутствовала)

=== INSERT (тестовые данные) ===
 Вставлено строк: 3

=== SELECT (все) ===
Получено строк: 3
{'id': 1, 'name': 'Alice', 'age': 19, 'avg_grade': 4.2, 'enrolled': True, 'created_at': datetime.datetime(2025, 10, 21, 15, 1, 11)}
{'id': 2, 'name': 'Bob', 'age': 21, 'avg_grade': 3.8, 'enrolled': True, 'created_at': datetime.datetime(2025, 10, 21, 15, 1, 11)}
{'id': 3, 'name': 'Charlie', 'age': 20, 'avg_grade': 4.9, 'enrolled': False, 'created_at': datetime.datetime(2025, 10, 21, 15, 1, 11)}

=== UPDATE (Bob → avg_grade=4.1) ===
 Обновлено строк: 1
Получено строк: 1
[{'id': 2, 'name': 'Bob', 'age': 21, 'avg_grade': 4.1, 'enrolled': True, 'created_at': datetime.datetime(2025, 10, 21, 15, 1, 11)}]

=== Экспорт в CSV ===
Получено строк: 3
Экспортировано в CSV: /content/students_export.csv
